In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tfkl
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm 

2021-11-18 14:26:08.881502: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [19]:
filepath = '../data/folclore-web/clean_dataset.txt'
#filepath = '../data/chamame-web/txts/chamame_dataset.txt'
f = open(filepath)
folclore_dataset = f.read()

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True) #default es word level encoding! 
tokenizer.fit_on_texts(folclore_dataset)

In [20]:
max_id = len(tokenizer.word_index) # Cantiad de caracteres distintos
print('Cantidad de caracteres diferentes: ', max_id)
dataset_size = tokenizer.document_count
print('Cantidad de caracteres en el dataset: ',dataset_size) # Cantidad de caracteres en el dataset
encoded = np.array(tokenizer.texts_to_sequences(folclore_dataset)) - 1

Cantidad de caracteres diferentes:  37
Cantidad de caracteres en el dataset:  1933003


In [4]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

2021-11-18 14:26:19.176291: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-18 14:26:19.231267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 14:26:19.232814: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7715GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-11-18 14:26:19.232928: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-18 14:26:19.246216: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-18 14:26:19.246365: I tensorflow/stream_executo

In [5]:
n_steps = 100 # largo de cada ventanita
window_length = n_steps + 1 # ya que el target es la entrada corrida 1 valor hacia el futuro
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

#batch_size = 256
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1,0], windows[:, 1:,0])) # (x, y)

# Las entradas categoricas usualmente se codifican, como one-hot o usando embedding. Probar de no hacerlo
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [6]:
#visualizo algunos datos. Las entradas son secuencias one-hot, y las salidas secuencias numericas! 
for i in dataset.take(1):
    print(i[0].shape)

(1, 100, 46)


2021-11-18 14:26:20.658836: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-18 14:26:20.678304: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


### Entrenamiento:
Defino el modelo, pero ahora necesito que al final de cada epoca se reseteen los estados ocultos antes de volver al inicio de una secuencia, es decir, para no perder estos estados (stateful). Para eso defino un callback que se encargue de aplicar este proceso al final de cada epoca. 


In [25]:
model = tf.keras.models.Sequential([
    tfkl.LSTM(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tfkl.LSTM(128, return_sequences=True, dropout=0.2),
    tfkl.TimeDistributed(tfkl.Dense(max_id, activation="softmax"))
    ])

In [26]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [17]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10, callbacks=[ResetStatesCallback()])

Epoch 1/10


2021-11-13 15:30:35.370091: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-11-13 15:30:35.638368: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-11-13 15:30:35.708428: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


     12/Unknown - 2s 5ms/step - loss: 3.5791

2021-11-13 15:30:35.922034: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


399/399 [==============================] - 4s 5ms/step - loss: 2.8598
Epoch 2/10
399/399 [==============================] - 2s 5ms/step - loss: 2.4555
Epoch 3/10
399/399 [==============================] - 2s 5ms/step - loss: 2.3459
Epoch 4/10
399/399 [==============================] - 2s 5ms/step - loss: 2.2861
Epoch 5/10
399/399 [==============================] - 2s 5ms/step - loss: 2.2360
Epoch 6/10
399/399 [==============================] - 2s 5ms/step - loss: 2.2058
Epoch 7/10
399/399 [==============================] - 2s 5ms/step - loss: 2.1704
Epoch 8/10
399/399 [==============================] - 2s 5ms/step - loss: 2.1399
Epoch 9/10
399/399 [==============================] - 2s 5ms/step - loss: 2.1212
Epoch 10/10
399/399 [==============================] - 2s 5ms/step - loss: 2.0963


In [73]:
model.save_weights('pesos/stateful.hdf5')

# Entrenar con baches

In [27]:
#filepath = '../data/folclore-web/folclore_dataset.txt'
filepath = '../data/chamame-web/txts/chamame_dataset.txt'
f = open(filepath)
folclore_dataset = f.read()

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True) #default es word level encoding! 
tokenizer.fit_on_texts(folclore_dataset)

max_id = len(tokenizer.word_index) # Cantiad de caracteres distintos
print('Cantidad de caracteres diferentes: ', max_id)
dataset_size = tokenizer.document_count
print('Cantidad de caracteres en el dataset: ',dataset_size) # Cantidad de caracteres en el dataset
encoded = np.array(tokenizer.texts_to_sequences(folclore_dataset)) - 1

Cantidad de caracteres diferentes:  46
Cantidad de caracteres en el dataset:  44401


In [28]:
train_size = dataset_size * 90 // 100
n_steps = 300 # largo de cada ventanita
window_length = n_steps + 1
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1,0], windows[:, 1:,0]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [30]:
model = tf.keras.models.Sequential([
    tfkl.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tfkl.GRU(128, return_sequences=True, dropout=0.2),
    tfkl.TimeDistributed(tfkl.Dense(max_id, activation="softmax"))
    ])

class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=1000, callbacks=[ResetStatesCallback()])

Epoch 1/1000
4/4 [==============================] - 2s 29ms/step - loss: 3.7958
Epoch 2/1000
4/4 [==============================] - 0s 29ms/step - loss: 3.6667
Epoch 3/1000
4/4 [==============================] - 0s 27ms/step - loss: 3.3409
Epoch 4/1000
4/4 [==============================] - 0s 25ms/step - loss: 3.1287
Epoch 5/1000
4/4 [==============================] - 0s 25ms/step - loss: 3.0172
Epoch 6/1000
4/4 [==============================] - 0s 25ms/step - loss: 3.0161
Epoch 7/1000
4/4 [==============================] - 0s 26ms/step - loss: 2.9728
Epoch 8/1000
4/4 [==============================] - 0s 26ms/step - loss: 2.9461
Epoch 9/1000
4/4 [==============================] - 0s 28ms/step - loss: 2.9365
Epoch 10/1000
4/4 [==============================] - 0s 26ms/step - loss: 2.9188
Epoch 11/1000
4/4 [==============================] - 0s 25ms/step - loss: 2.9060
Epoch 12/1000
4/4 [==============================] - 0s 25ms/step - loss: 2.8908
Epoch 13/1000
4/4 [==================

4/4 [==============================] - 0s 24ms/step - loss: 2.0585
Epoch 103/1000
4/4 [==============================] - 0s 25ms/step - loss: 2.0549
Epoch 104/1000
4/4 [==============================] - 0s 25ms/step - loss: 2.0476
Epoch 105/1000
4/4 [==============================] - 0s 28ms/step - loss: 2.0495
Epoch 106/1000
4/4 [==============================] - 0s 27ms/step - loss: 2.0427
Epoch 107/1000
4/4 [==============================] - 0s 25ms/step - loss: 2.0429
Epoch 108/1000
4/4 [==============================] - 0s 24ms/step - loss: 2.0366
Epoch 109/1000
4/4 [==============================] - 0s 25ms/step - loss: 2.0341
Epoch 110/1000
4/4 [==============================] - 0s 26ms/step - loss: 2.0245
Epoch 111/1000
4/4 [==============================] - 0s 31ms/step - loss: 2.0287
Epoch 112/1000
4/4 [==============================] - 0s 30ms/step - loss: 2.0288
Epoch 113/1000
4/4 [==============================] - 0s 26ms/step - loss: 2.0169
Epoch 114/1000
4/4 [===========

4/4 [==============================] - 0s 27ms/step - loss: 1.7896
Epoch 203/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.7754
Epoch 204/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.7902
Epoch 205/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.7902
Epoch 206/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.7943
Epoch 207/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.7853
Epoch 208/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.7783
Epoch 209/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.7778
Epoch 210/1000
4/4 [==============================] - 0s 24ms/step - loss: 1.7777
Epoch 211/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.7727
Epoch 212/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.7706
Epoch 213/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.7687
Epoch 214/1000
4/4 [===========

4/4 [==============================] - 0s 25ms/step - loss: 1.6172
Epoch 303/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.6249
Epoch 304/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.6179
Epoch 305/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.6143
Epoch 306/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.6227
Epoch 307/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.6231
Epoch 308/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.6121
Epoch 309/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.6102
Epoch 310/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.6057
Epoch 311/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.6058
Epoch 312/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.6095
Epoch 313/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.6078
Epoch 314/1000
4/4 [===========

4/4 [==============================] - 0s 25ms/step - loss: 1.4983
Epoch 403/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.4998
Epoch 404/1000
4/4 [==============================] - 0s 39ms/step - loss: 1.5052
Epoch 405/1000
4/4 [==============================] - 0s 30ms/step - loss: 1.4917
Epoch 406/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.5094
Epoch 407/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.5119
Epoch 408/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.4955
Epoch 409/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.4955
Epoch 410/1000
4/4 [==============================] - 0s 38ms/step - loss: 1.4934
Epoch 411/1000
4/4 [==============================] - 0s 39ms/step - loss: 1.4952
Epoch 412/1000
4/4 [==============================] - 0s 37ms/step - loss: 1.4952
Epoch 413/1000
4/4 [==============================] - 0s 36ms/step - loss: 1.4911
Epoch 414/1000
4/4 [===========

4/4 [==============================] - 0s 25ms/step - loss: 1.3966
Epoch 503/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.4067
Epoch 504/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.4026
Epoch 505/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.4189
Epoch 506/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.4154
Epoch 507/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.4109
Epoch 508/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.4012
Epoch 509/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3948
Epoch 510/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.3904
Epoch 511/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.4004
Epoch 512/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.4026
Epoch 513/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.3941
Epoch 514/1000
4/4 [===========

4/4 [==============================] - 0s 28ms/step - loss: 1.3432
Epoch 603/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.3324
Epoch 604/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.3480
Epoch 605/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3353
Epoch 606/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.3481
Epoch 607/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3418
Epoch 608/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.3443
Epoch 609/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.3373
Epoch 610/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3232
Epoch 611/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3340
Epoch 612/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3319
Epoch 613/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.3282
Epoch 614/1000
4/4 [===========

4/4 [==============================] - 0s 25ms/step - loss: 1.2889
Epoch 703/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2863
Epoch 704/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.2706
Epoch 705/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.2771
Epoch 706/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.2975
Epoch 707/1000
4/4 [==============================] - 0s 24ms/step - loss: 1.2737
Epoch 708/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2792
Epoch 709/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2699
Epoch 710/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.2835
Epoch 711/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.2833
Epoch 712/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.2841
Epoch 713/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2821
Epoch 714/1000
4/4 [===========

4/4 [==============================] - 0s 25ms/step - loss: 1.2270
Epoch 803/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2512
Epoch 804/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2350
Epoch 805/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.2421
Epoch 806/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.2318
Epoch 807/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2354
Epoch 808/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2190
Epoch 809/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2451
Epoch 810/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.2287
Epoch 811/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.2171
Epoch 812/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.2365
Epoch 813/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.2279
Epoch 814/1000
4/4 [===========

4/4 [==============================] - 0s 27ms/step - loss: 1.2153
Epoch 903/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.2016
Epoch 904/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.1919
Epoch 905/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.1948
Epoch 906/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.2041
Epoch 907/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.2006
Epoch 908/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.1947
Epoch 909/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.1945
Epoch 910/1000
4/4 [==============================] - 0s 25ms/step - loss: 1.1997
Epoch 911/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.1881
Epoch 912/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.1846
Epoch 913/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.1810
Epoch 914/1000
4/4 [===========

In [12]:
#model.save_weights('pesos/1000e_200steps_t0.3.hdf5')
#model.load_weights('pesos/1000e_200steps_t0.3.hdf5')

### Usando el modelo
Hay que hacer un pre-procesamiento para usar este modelo. Lo mas simple seria tomar una entrada de texto, hacer el one-hot encoding, pasarlo al modelo para hacer prediccion (el modelo recibe secuencias de cualquier largo!), tomar la salida del modelo (que es un numero, un codigo o ID) y convertirla a caracter a traves del tokenizer.

In [15]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["rin"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'd'

# Generar Texto!
Al igual que en los ejemplos de series numéricas, puedo predecir varios valores futuros yendo de uno a la vez, y sumando este valor al cuerpo de valores de entrada a medida que hago predicciones. En generación de texto esta metodología lleva a repetir palabras u oraciones. Otra alternativa mejor es elegir el caracter siguiente de manera aleatoria, con una probabilidad igual a la estimada por el modelo. Esto genera texto mas diverso e interesante.
Para hacer esto, dependemos de la función **tf.random.categorical()**. Esta funcion devuelve muestras random de un cierto numero de clases, dado las probabilidades de cada clase (logits). Para tener mas control sobre la diversidad del texto generado, se pueden dividir los logits por un número denominado "temperatura" que se puede ajustar. Con un valor de temperatura = 0, se favorecen los caracteres de alta probabilidad, y con una temperatura cerca de 1 se les da a todos el mismo peso. 

In [16]:
def simple_gen(text, n_chars):    
    for _ in tqdm(range(n_chars)):
        X_new = preprocess([text])
        Y_pred = model.predict_classes(X_new)
        next_char = tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]
        text += next_char
    return text

In [17]:
print(simple_gen("rio", 200))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:15<00:00, 32.95it/s]

rio te quiso 
y el sol 
jamás conseguirás. 

no piensas que no se ha ido 
de verdes mi tantar 
la bajada ver de cubar la luna 
de tara siempre se rubuda  
de los ásporas de mi provincia 
porá la luna es un camanotero 
alla en el resabatido 
en la entara 
por quien se va hay quiero partida. 


cuando en la espuma caña a 
por el medio se río un instante 
y soñarrerá.  


la tierra de dicha y paz. 


allí donde el cielo se puede tus calles de tierra, tacuaral y las jazmines florecidos. y es coriño des


In [31]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=400, temperature=1):
    for _ in tqdm(range(n_chars)):
        text += next_char(text, temperature)
    return text

In [33]:
print(complete_text("Tarde", temperature=0.50))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:11<00:00, 34.68it/s]

Tarde de la noche 
noches de mi provincia 
porá la luna la arena 
bajó con su insuante 
mi abando en la aresa de tu midar 
allá donde sobre la espala 
el alma del correntino   
tu nombre grande no morirá.   


perdón corrientes abardecir 
con su guitarra me cuntar 
los campos del corazón. 


al endiviendo la sombra 
en al alma del correntino   
muero con este canto 
con su fugaza de los pajosas  
la no
